<a href="https://colab.research.google.com/github/girotodenis/python/blob/master/relatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture

import numpy as np
import pandas as pd
from datetime import datetime
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [0]:
data_i = datetime(2020, 5, 1)
data_f = datetime(2020, 5, 22)
#nome_atribuido = 'Denis Silva Giroto'
nome_atribuido = 'todos'

In [0]:
%%capture

df = pd.read_excel('/content/ePol.xlsx')
df = df[(df['Progresso']!='Não iniciada')]
df['Data de início'].fillna(df['Criado em'], inplace=True)
df['Data de conclusão'].fillna(df['Concluído em'], inplace=True)
df['Data de início'] = pd.to_datetime(df['Data de início'], format='%d/%m/%Y')
df['Data de conclusão'] = pd.to_datetime(df['Data de conclusão'], format='%d/%m/%Y')
df['Criado em'] = pd.to_datetime(df['Criado em'], format='%d/%m/%Y')
df['Concluído em'] = pd.to_datetime(df['Concluído em'], format='%d/%m/%Y')
df['Atribuído a'].fillna(df['Criado por'], inplace=True)
df = df[['Nome da tarefa','Nome do Bucket','Progresso','Atribuído a', 'Criado por', 'Criado em', 'Data de início', 'Data de conclusão']]
data_minima = df['Data de início'].min()
data_maxima = df['Data de conclusão'].max()

In [0]:
%%capture

sequencia_dias = pd.DataFrame(data={'Dia':[]})
sequencia_dias['Dia'] = pd.date_range(start=data_minima,end=data_maxima, name="dia")
df_completo = pd.DataFrame(columns=df.columns)
df_completo['Dia'] = None
for data_corrente in sequencia_dias['Dia'].values: 
  x = df[ ((df['Data de início']<=data_corrente) & (df['Progresso']!='Concluída')) | ((df['Data de início']<=data_corrente) & (df['Data de conclusão']>=data_corrente) ) ] 
  x['Dia'] = data_corrente
  x['count'] = 1
  x['Progresso dia'] = x['Progresso']
  df_completo = df_completo.append(x, ignore_index=True) 
df_completo['Progresso dia'].unique()
df_completo.loc[((df_completo['Progresso']=='Concluída')&( df_completo['Data de conclusão'] > df_completo['Dia'] )), 'Progresso dia'] = 'Em andamento' 



In [0]:
%%capture

qtd_demanda = df_completo[['Dia', 'Atribuído a', 'count']]
qtd_demanda.sort_values(by=['Atribuído a'], inplace=True)
qtd_demanda = qtd_demanda.groupby(['Dia', 'Atribuído a']).count()
qtd_demanda = qtd_demanda.reset_index()
qtd_demanda['Dia'] = pd.to_datetime(qtd_demanda['Dia'], format='%d/%m/%Y')

In [0]:
##%%capture

df_x = qtd_demanda[ ((qtd_demanda['Dia']>=data_i) & (qtd_demanda['Dia']<=data_f) ) ]
df_x['Dia'] = df_x['Dia'].astype(str)


fig, ax = plt.subplots(figsize = (25,7))
values = ['g','b','r','y']
i=0
for nome in df_x['Atribuído a'].unique():
  if nome_atribuido == 'todos' or nome_atribuido == nome:
    color = values[i]
    filtro = df_x[(df_x['Atribuído a']==nome)]
    x = filtro['Dia']
    y = filtro['count']
    ax.plot(x, y, 'k--', label = nome, color=color,  marker = 'o') 
    i+=1

x=plt.xticks(rotation = 90) 
x=plt.tight_layout() 
x=plt.legend(loc = 1)
x=plt.title('Gráfico de Controle', size = 30, weight = 'bold', color = '#006400', style = 'italic') 
x=plt.xlabel('Dias', size = 10)
x=plt.ylabel('Qt de tarefas', size = 10)
x=plt.grid() 




In [0]:

print('------------------------------------')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
df_x.sort_values(by=['Atribuído a'], inplace=True, ascending=True)

for nome in df_x['Atribuído a'].unique():
  if nome_atribuido == 'todos' or nome_atribuido == nome:
    print(df_x[df_x['Atribuído a']==nome])
    print('------------------------------------------------')

print('------------------------------------')

In [0]:
barra = {'Atribuído a':[], 'Tarefas':[], 'Concluidos':[] }
for nome in df_x['Atribuído a'].unique():
  if nome_atribuido == 'todos' or nome_atribuido == nome:
    print(nome)
    tarefas = df_completo[(df_completo['Atribuído a']==nome) & (df_completo['Dia']>=data_i) & (df_completo['Dia']<=data_f)]
    tarefas_concluidas = df_completo[(df_completo['Atribuído a']==nome) & (df_completo['Dia']>=data_i) & (df_completo['Dia']<=data_f) & (df_completo['Progresso dia']=='Concluída' )]
    tarefas = tarefas['Nome da tarefa'].unique()
    tarefas_concluidas = tarefas_concluidas['Nome da tarefa'].unique()
    print('quantidade de tarefas no periodo', len(tarefas))
    print('tarefas:', tarefas)
    print('tarefas concluidas:', tarefas_concluidas)
    barra['Atribuído a'].append(nome)
    barra['Tarefas'].append(len(tarefas))
    barra['Concluidos'].append(len(tarefas_concluidas))
    print('------------------------------------------------')



In [0]:
df_barra = pd.DataFrame(barra)
pp = df_barra.plot
#x = pp.grid()
ax = pp.barh(x='Atribuído a', y=['Tarefas','Concluidos'], figsize=(15,7))
ax.grid('on', which='minor', axis='x' )
ax.grid('on', which='major', axis='x' )
ax.set_alpha(0.8)
ax.set_xticks(barra['Tarefas'])
for i in ax.patches:
  ax.text( 0 if i.get_width()-1<0 else i.get_width()-1, i.get_y()+.1, \
            str( i.get_width() ), fontsize=15,color='white')